# Iterative random shuffling image denoising VS [cv2.fastNlMeansDenoising()](https://docs.opencv.org/3.4/d5/d69/tutorial_py_non_local_means.html) (lena)

## Imports

In [ ]:
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
#from ipywidgets import *
import cv2
import time
#import kernels
from skimage import io as skimage_io
from skimage.metrics import structural_similarity as ssim
from scipy import stats
import math
from color_transforms import YCoCg as YUV #pip install "color_transforms @ git+https://github.com/vicente-gonzalez-ruiz/color_transforms"

In [ ]:
import denoising.image.OF_iterative_random_shuffling as random_denoising

In [ ]:
import logging

## Input image

In [ ]:
img = skimage_io.imread("http://www.hpca.ual.es/~vruiz/images/lena.png")

In [ ]:
plt.imshow(img)

## Add noise

In [ ]:
mean = 0
var = 1000
sigma = var**0.5
noise = np.random.normal(mean,sigma,img.shape).reshape(img.shape)
noisy_img = np.clip(a=img.astype(np.float32) + noise, a_min=0, a_max=255).astype(np.uint8)

In [ ]:
plt.imshow(noisy_img)

## Low filtering level

In [ ]:
denoised_img_fastNlMeans = cv2.fastNlMeansDenoisingColored(src=noisy_img, dst=None, h=11, hColor=6, templateWindowSize=7, searchWindowSize=21)

In [ ]:
denoiser = random_denoising.Filter_Color_Image(l=3, w=11, poly_sigma=0.75, verbosity=logging.WARNING)
denoised_img, _ = denoiser.filter(noisy_image=noisy_img, RS_sigma=1.0, GT=img, N_iters=25)
denoised_img = denoised_img.astype(np.uint8)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(16, 32))
axs[0].imshow(noisy_img)
axs[0].set_title(f"Noisy {ssim(YUV.from_RGB(img)[..., 0], YUV.from_RGB(noisy_img)[..., 0])}")
axs[1].imshow(denoised_img_fastNlMeans)
axs[1].set_title(f"fastNlMeans {ssim(YUV.from_RGB(img)[..., 0], YUV.from_RGB(denoised_img_fastNlMeans)[..., 0])}")
axs[2].imshow(denoised_img)
axs[2].set_title(f"RSIID {ssim(YUV.from_RGB(img)[..., 0], YUV.from_RGB(denoised_img)[..., 0])}")
fig.tight_layout()
plt.show()

## Medium filtering level

In [ ]:
denoised_img_fastNlMeans = cv2.fastNlMeansDenoisingColored(src=noisy_img, dst=None, h=17, hColor=17, templateWindowSize=7, searchWindowSize=21)

In [ ]:
denoiser = random_denoising.Filter_Color_Image(l=3, w=9, poly_sigma=1.0, verbosity=logging.WARNING)
denoised_img, _ = denoiser.filter(noisy_image=noisy_img, RS_sigma=1.0, GT=img, N_iters=25)
denoised_img = denoised_img.astype(np.uint8)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(16, 32))
axs[0].imshow(noisy_img)
axs[0].set_title(f"Noisy {ssim(YUV.from_RGB(img)[..., 0], YUV.from_RGB(noisy_img)[..., 0])}")
axs[1].imshow(denoised_img_fastNlMeans)
axs[1].set_title(f"fastNlMeans {ssim(YUV.from_RGB(img)[..., 0], YUV.from_RGB(denoised_img_fastNlMeans)[..., 0])}")
axs[2].imshow(denoised_img)
axs[2].set_title(f"RSIID {ssim(YUV.from_RGB(img)[..., 0], YUV.from_RGB(denoised_img)[..., 0])}")
fig.tight_layout()
plt.show()

## High filtering level

In [ ]:
denoised_img_fastNlMeans = cv2.fastNlMeansDenoisingColored(src=noisy_img, dst=None, h=25, hColor=25, templateWindowSize=7, searchWindowSize=21)

In [ ]:
denoiser = random_denoising.Filter_Color_Image(l=3, w=7, poly_sigma=1.25, verbosity=logging.WARNING)
denoised_img, _ = denoiser.filter(noisy_image=noisy_img, RS_sigma=1.0, GT=img, N_iters=25)
denoised_img = denoised_img.astype(np.uint8)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(16, 32))
axs[0].imshow(noisy_img)
axs[0].set_title(f"Noisy {ssim(YUV.from_RGB(img)[..., 0], YUV.from_RGB(noisy_img)[..., 0])}")
axs[1].imshow(denoised_img_fastNlMeans)
axs[1].set_title(f"fastNlMeans {ssim(YUV.from_RGB(img)[..., 0], YUV.from_RGB(denoised_img_fastNlMeans)[..., 0])}")
axs[2].imshow(denoised_img)
axs[2].set_title(f"RSIID {ssim(YUV.from_RGB(img)[..., 0], YUV.from_RGB(denoised_img)[..., 0])}")
fig.tight_layout()
plt.show()

## [Measurement of the denoising](http://faculty.ucmerced.edu/mhyang/papers/iccv13_denoise.pdf)

In [ ]:
def get_denoising_quality(img, denoised_img):
    img = YUV.from_RGB(img)[..., 0]
    denoised_img = YUV.from_RGB(denoised_img)[..., 0]
    diff_img = (img - denoised_img).astype(np.uint8)
    _, N = ssim(img, diff_img, full=True)
    #print(_)
    _, P = ssim(img, denoised_img.astype(np.uint8), full=True)
    #print(_)
    quality, _ = stats.pearsonr(N.flatten(), P.flatten())
    #print("quality", quality)
    if math.isnan(quality):
        return 0.0
    else:
        return -quality

## Optimization of the parameters of cv2.fastNlMeansDenoising()

In [ ]:
# Optimizing h
prev_quality = -1
for i in range(3, 20):
    #denoised_img = cv2.fastNlMeansDenoising(img, None, 11, 6, 7, 21)
    denoised_img = cv2.fastNlMeansDenoisingColored(src=noisy_img, dst=None, h=i, hColor=i, templateWindowSize=7, searchWindowSize=21)
    quality = get_denoising_quality(noisy_img, denoised_img)
    if prev_quality > quality:
        optimal_h = i - 1
        break
    plt.imshow(denoised_img)
    plt.title(f"{i} {quality}")
    plt.show()
    prev_quality = quality

print("optimal h", optimal_h)

In [ ]:
# Optimizing templateWindowSize
prev_quality = -1
for i in range(2, 22):
    #denoised_img = cv2.fastNlMeansDenoising(img, None, 11, 6, 7, 21)
    denoised_img = cv2.fastNlMeansDenoisingColored(src=noisy_img, dst=None, h=5, hColor=5, templateWindowSize=i, searchWindowSize=21)
    quality = get_denoising_quality(noisy_img, denoised_img)
    if prev_quality > quality:
        optimal_templateWindowSize = i - 1
        break
    plt.imshow(denoised_img)
    plt.title(f"{i} {quality}")
    plt.show()
    prev_quality = quality

print("optimal templateWindowSize", optimal_templateWindowSize)

In [ ]:
# Optimizing searchWindowSize
prev_quality = -1
for i in range(3, 33):
    denoised_img = cv2.fastNlMeansDenoisingColored(src=noisy_img, dst=None, h=5, hColor=5, templateWindowSize=13, searchWindowSize=i)
    quality = get_denoising_quality(noisy_img, denoised_img)
    if prev_quality > quality:
        optimal_searchWindowSize = i - 1
        break
    plt.imshow(denoised_img)
    plt.title(f"{i} {quality}")
    plt.show()
    prev_quality = quality

print("optimal searchWindowSize", optimal_searchWindowSize)

## Optimization of the parameters of RSIID

In [ ]:
# Optimizing RS_sigma
prev_quality = -1
for i in range(7, 30):
    _i = i/10
    denoiser = random_denoising.Filter_Color_Image(l=3, w=7, poly_sigma=1.0, verbosity=logging.WARNING)
    denoised_img, _ = denoiser.filter(noisy_image=noisy_img, RS_sigma=_i, GT=img, N_iters=20)
    denoised_img = denoised_img.astype(np.uint8)
    quality = get_denoising_quality(noisy_img, denoised_img)
    if prev_quality > quality:
        optimal_RS_sigma = _i - 0.1
        break
    plt.imshow(denoised_img)
    plt.title(f"{_i} {quality}")
    plt.show()
    prev_quality = quality

print("optimal RS_sigma", optimal_RS_sigma)

In [ ]:
# Optimizing poly_sigma
prev_quality = -1
for i in range(5, 30):
    _i = i/10
    denoiser = random_denoising.Filter_Color_Image(l=3, w=5, poly_sigma=_i, verbosity=logging.WARNING)
    denoised_img, _ = denoiser.filter(noisy_image=noisy_img, RS_sigma=1.6, GT=img, N_iters=20)
    denoised_img = denoised_img.astype(np.uint8)
    quality = get_denoising_quality(noisy_img, denoised_img)
    if prev_quality > quality:
        optimal_poly_sigma = _i - 0.1
        break
    plt.imshow(denoised_img)
    plt.title(f"{_i} {quality}")
    plt.show()
    prev_quality = quality

print("optimal poly_sigma", optimal_poly_sigma)

In [ ]:
# Optimizing w
prev_quality = -1
for i in range(4, 30):
    denoiser = random_denoising.Filter_Color_Image(l=3, w=i, poly_sigma=0.6, verbosity=logging.WARNING)
    denoised_img, _ = denoiser.filter(noisy_image=noisy_img, RS_sigma=1.6, GT=img, N_iters=20)
    denoised_img = denoised_img.astype(np.uint8)
    quality = get_denoising_quality(noisy_img, denoised_img)
    if prev_quality > quality:
        optimal_w = i - 1
        break
    plt.imshow(denoised_img)
    plt.title(f"{i} {quality}")
    plt.show()
    prev_quality = quality

print("optimal w", optimal_w)

In [ ]:
# Optimizing N_iters
prev_quality = -1
for i in range(3, 30):
    denoiser = random_denoising.Filter_Color_Image(l=3, w=5, poly_sigma=0.6, verbosity=logging.WARNING)
    denoised_img, _ = denoiser.filter(noisy_image=noisy_img, RS_sigma=1.6, GT=img, N_iters=i)
    denoised_img = denoised_img.astype(np.uint8)
    quality = get_denoising_quality(noisy_img, denoised_img)
    if prev_quality > quality:
        optimal_N_iters = i - 1
        break
    plt.imshow(denoised_img)
    plt.title(f"{i} {quality}")
    plt.show()
    prev_quality = quality

print("optimal N_iters", optimal_N_iters)

## Comparison using the optimal configurations

In [ ]:
denoised_img_fastNlMeans = cv2.fastNlMeansDenoisingColored(src=noisy_img, dst=None, h=5, hColor=5, templateWindowSize=13, searchWindowSize=31)

In [ ]:
denoiser = random_denoising.Filter_Color_Image(l=3, w=5, poly_sigma=0.6, verbosity=logging.WARNING)
denoised_img, _ = denoiser.filter(noisy_image=noisy_img, RS_sigma=1.6, GT=img, N_iters=8)
denoised_img = denoised_img.astype(np.uint8)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(16, 32))
axs[0].imshow(noisy_img)
axs[0].set_title(f"Noisy {ssim(YUV.from_RGB(img)[..., 0], YUV.from_RGB(noisy_img)[..., 0])}")
axs[1].imshow(denoised_img_fastNlMeans)
axs[1].set_title(f"fastNlMeans {ssim(YUV.from_RGB(img)[..., 0], YUV.from_RGB(denoised_img_fastNlMeans)[..., 0])}")
axs[2].imshow(denoised_img)
axs[2].set_title(f"RSIID {ssim(YUV.from_RGB(img)[..., 0], YUV.from_RGB(denoised_img)[..., 0])}")
fig.tight_layout()
plt.show()